In [16]:
import pandas as pd 
import numpy as np

import requests
from bs4 import BeautifulSoup

from time import sleep 
from random import randint

In [17]:
headers ={ "Accept-Language": "en-US,en;q=0.5"}

In [18]:
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'

response = requests.get(url)

In [19]:
soup = BeautifulSoup(response.content, "html.parser")

In [20]:
pages = np.arange(1,1000,100)

In [21]:
movie_name = []
time =[]
year =[]
rating =[]
metascore =[]
votes =[]
gross =[]
description=[]

for page in pages:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    
    movie_data = soup.findAll("div",attrs ={ 'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    
    for item in movie_data:
        name = item.h3.a.text
        movie_name.append(name)
        
        year_of_release = item.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(year_of_release)
        
        runtime = item.p.find("span", class_ = 'runtime').text
        time.append(runtime)
        
        rate = item.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = item.find('span', class_ = "metascore").text if item.find('span', class_ = "metascore") else ""
        metascore.append(meta)
        
        
        value = item.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 else ''
        gross.append(grosses)
        
        
        describe = item.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else ''
        description.append(description_)


In [22]:
movie_df =pd.DataFrame({
    
                       "Movie Name": movie_name,
                       "Year of Release": year, 
                       "Watch Time": time,
                       "Movie Rating": rating, 
                       "Metascore of movie": metascore,
                       "Votes": votes, 
                       "Gross Income": gross,
                        "Description": description
                       })

In [23]:
movie_df.head(5)

,Movie Name,Year of Release,Watch Time,Movie Rating,Metascore of movie,Votes,Gross Income,Description
0,The Shawshank Redemption,(1994),142 min,9.3,82,"2,798,876",$28.34M,"Over the course of several years, two convicts..."
1,The Godfather,(1972),175 min,9.2,100,"1,949,996",$134.97M,"Don Vito Corleone, head of a mafia family, dec..."
2,The Dark Knight,(2008),152 min,9.0,84,"2,780,083",$534.86M,When the menace known as the Joker wreaks havo...
3,Schindler's List,(1993),195 min,9.0,95,"1,407,123",$96.90M,"In German-occupied Poland during World War II,..."
4,The Godfather Part II,(1974),202 min,9.0,90,"1,324,390",$57.30M,The early life and career of Vito Corleone in ...


In [24]:
movie_df.to_excel("Top 1000 movies from IMDb.xlsx")
